In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

train_dir = "/content/drive/MyDrive/monument/Monuments.v17i.tensorflow/train"
test_dir = "/content/drive/MyDrive/monument/Monuments.v17i.tensorflow/test"

if not os.path.exists(train_dir):
    raise FileNotFoundError(f"❌ Train directory not found: {train_dir}")
if not os.path.exists(test_dir):
    raise FileNotFoundError(f"❌ Test directory not found: {test_dir}")

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3]
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

train_dataset = train_datagen.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="sparse", shuffle=True
)

test_dataset = test_datagen.flow_from_directory(
    test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="sparse", shuffle=False
)

print(f" Classes: {train_dataset.class_indices}")

base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
output_layer = Dense(len(train_dataset.class_indices), activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output_layer)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

EPOCHS = 15
model.fit(train_dataset, validation_data=test_dataset, epochs=EPOCHS)

model_path = "/content/drive/MyDrive/monument/model/monument_model_mobilenetv2_aug.keras"
model.save(model_path)
print(f" Model saved at: {model_path}")

Found 716 images belonging to 17 classes.
Found 69 images belonging to 17 classes.
 Classes: {'Agra Fort': 0, 'Agrasen Ki Baoli': 1, 'Gurudwara Bangla Sahib': 2, 'Humayu Tomb': 3, 'India gate': 4, 'Isa Khan Niyazi-s tomb': 5, 'Itmad-Ud-Daulah-s Tomb': 6, 'Jama Mashjid': 7, 'Jantar Mantar': 8, 'Lotus Temple': 9, 'Mutiny Memorial': 10, 'Qila-i-Kuhna mosque': 11, 'Qutub Minar': 12, 'Quwwat ul-Islam Mosque': 13, 'Rashtrapati Bhavan': 14, 'Red Fort': 15, 'Taj Mahal': 16}
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 311s 13s/step - accuracy: 0.2652 - loss: 2.5197 - val_accuracy: 0.7246 - val_loss: 1.2060
Epoch 2/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 76s 2s/step - accuracy: 0.7796 - loss: 0.9731 - val_accuracy: 0.8551 - val_loss: 0.5961
Epoch 3/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 79s 2s/step - accuracy: 0.9048 - loss: 0.4672 - val_accuracy: 0.8841 - val_loss: 0.4679
Epoch 4/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - accuracy: 0.9184 - loss: 0.3189 - val_accuracy: 0.8841 - val_loss: 0.4277
Epoch 5/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 52s 2s/step - accuracy: 0.9465 - loss: 0.2429 - val_accuracy: 0.9420 - val_loss: 0.3013
Epoch 6/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.9589 - loss: 0.1939 - val_accuracy: 0.9130 - val_loss: 0.3111
Epoch 7/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.9584 - loss: 0.1745 - val_accuracy: 0.9275 - val_loss: 0.2977
Epoch 8/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 78s 2s/step - accuracy: 0.9787 - loss: 0.0991 - val_accuracy: 0.9420 - val_los

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

model_path = "/content/drive/MyDrive/monument/model/monument_model_mobilenetv2_aug.keras"
model = load_model(model_path)

class_labels = ['Agra Fort', 'Agrasen Ki Baoli', 'Gurudwara Bangla Sahib', 'Humayu Tomb', 'India gate',
                'Isa Khan Niyazi-s tomb', 'Itmad-Ud-Daulah-s Tomb', 'Jama Mashjid', 'Jantar Mantar',
                'Lotus Temple', 'Mutiny Memorial', 'Qila-i-Kuhna mosque', 'Qutub Minar',
                'Quwwat ul-Islam Mosque', 'Rashtrapati Bhavan', 'Red Fort', 'Taj Mahal']

img_path = "/content/Agrasen-Ki-Baoli-Haunted-12-1024x682.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

pred = model.predict(img_array)
pred_class = np.argmax(pred)
pred_label = class_labels[pred_class]

print(f" Predicted Class: {pred_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
 Predicted Class: Agrasen Ki Baoli
